In [1]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
# nltk.download('punkt')
import tensorflow_hub as hub
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

from nltk.stem import WordNetLemmatizer

In [2]:
# load the dataset
data = pd.read_csv('dataset.csv')
data

,id,question,desired_answer,student_answer,score_me,score_other,score_avg
0,1.10,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,High risk problems are address in the prototyp...,4.0,3.0,3.5
1,1.10,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,To simulate portions of the desired final prod...,5.0,5.0,5.0
2,1.10,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,A prototype program simulates the behaviors of...,5.0,3.0,4.0
3,1.10,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,Defined in the Specification phase a prototype...,5.0,5.0,5.0
4,1.10,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,It is used to let the users have a first idea ...,3.0,3.0,3.0
...,...,...,...,...,...,...,...
2711,12.11,How many steps does it take to search a node i...,The height of the tree.,"it takes at most h steps, where h is the heigh...",5.0,5.0,5.0
2712,12.11,How many steps does it take to search a node i...,The height of the tree.,actually i dont know,0.0,0.0,0.0
2713,12.11,How many steps does it take to search a node i...,The height of the tree.,height height height height height height height,0.0,0.0,0.0
2714,12.11,How many steps does it take to search a node i...,The height of the tree.,tree tree tree tree,0.0,0.0,0.0


In [3]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # convert to lowercase
    text = text.lower()
    # remove punctuation and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # remove stopwords and lemmatize
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    text = ' '.join(tokens)
    return text

In [4]:
# preprocess the data
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X = np.array([embed([preprocess_text(str(q) + ' ' + str(a))]).numpy().flatten() for q, a in zip(data['question'], data['student_answer'])])
y = data['score_avg']

In [5]:
X

array([[ 0.05768189,  0.01441896, -0.03444666, ...,  0.00758925,
         0.00898315,  0.02601086],
       [-0.0123023 , -0.06516756,  0.02217915, ..., -0.028247  ,
         0.02213215,  0.00750732],
       [ 0.06155297, -0.02784808,  0.03194202, ...,  0.01315192,
         0.01880543,  0.03179809],
       ...,
       [-0.04257369, -0.01115208,  0.00029492, ..., -0.01791414,
        -0.02747966, -0.03977058],
       [-0.00857453, -0.01678239,  0.03052462, ...,  0.01601931,
        -0.06068902, -0.01925499],
       [-0.02729181, -0.01601752,  0.01836829, ...,  0.05408085,
        -0.06122277, -0.001444  ]], dtype=float32)

In [6]:
y

0       3.5
1       5.0
2       4.0
3       5.0
4       3.0
       ... 
2711    5.0
2712    0.0
2713    0.0
2714    0.0
2715    0.5
Name: score_avg, Length: 2716, dtype: float64

In [8]:
with open('X_embedded.npy', 'wb') as f:
    np.save(f, X)
with open('y_target.npy', 'wb') as f:
    np.save(f, y)
